# Curriculum Acquire code

## Making HTTP Requests

- The way we interact with web sites and web servers (including RESTful JSON APIs) is through HTTP **requests** and **responses**.

- We can use the `requests` library to make http requests. This is somewhat the same as visiting a url in your browser, except that we can interact with the responses programatically in python.

In [1]:
import requests


We will use the `get` function from `requests` and pass it a url:



In [2]:
response = requests.get('http://request-inspector.glitch.me/')
response

<Response [200]>

<p>We get back a python object that represents an HTTP response.</p>
<p>The response object has several interesting properties:</p>
<ul>
<li><code>.ok</code>: a boolean that indicates that the response was successful (the server sent back a 200 response code)</li>
<li><code>.status_code</code>: a number indicating the HTTP response status code </li>
<li><code>.text</code>: the raw response text</li>
</ul>

In [3]:
response.ok


True

In [4]:
response.status_code


200

In [5]:
response.text


'{"method":"GET","query":{},"body":{}}'

<p>In this case, we see a string that contains HTML. HTML is what makes up web pages that are intended for humans to read. If you go to http://example.com, you'll see what the HTML in the above response looks like when rendered. Some other endpoints on the internet return JSON, which is usually intended to be worked with programatically.</p>


<h2 id="example-json-api">Example JSON API</h2>
<p>For an example of a JSON api, we'll interact with the <a href="https://aphorisms.glitch.me/">a quote generator</a>.</p>


In [6]:
url = 'https://swapi.dev/api/people/5'
response = requests.get(url)
print(response.text)

{"name":"Leia Organa","height":"150","mass":"49","hair_color":"brown","skin_color":"light","eye_color":"brown","birth_year":"19BBY","gender":"female","homeworld":"https://swapi.dev/api/planets/2/","films":["https://swapi.dev/api/films/1/","https://swapi.dev/api/films/2/","https://swapi.dev/api/films/3/","https://swapi.dev/api/films/6/"],"species":[],"vehicles":["https://swapi.dev/api/vehicles/30/"],"starships":[],"created":"2014-12-10T15:20:09.791000Z","edited":"2014-12-20T21:17:50.315000Z","url":"https://swapi.dev/api/people/5/"}


<p>Here we see that the repsonse we got back contains a JSON object (we could also verify this by visiting the URL in a web browser).</p>
<p>Since the response is JSON, we can use the <code>.json</code> method on the response object to get a data structure we can work with:</p>

In [7]:
data = response.json()
print(type(data))
data

<class 'dict'>


{'name': 'Leia Organa',
 'height': '150',
 'mass': '49',
 'hair_color': 'brown',
 'skin_color': 'light',
 'eye_color': 'brown',
 'birth_year': '19BBY',
 'gender': 'female',
 'homeworld': 'https://swapi.dev/api/planets/2/',
 'films': ['https://swapi.dev/api/films/1/',
  'https://swapi.dev/api/films/2/',
  'https://swapi.dev/api/films/3/',
  'https://swapi.dev/api/films/6/'],
 'species': [],
 'vehicles': ['https://swapi.dev/api/vehicles/30/'],
 'starships': [],
 'created': '2014-12-10T15:20:09.791000Z',
 'edited': '2014-12-20T21:17:50.315000Z',
 'url': 'https://swapi.dev/api/people/5/'}

<p>Now we have a dictionary that we can work with.</p>
<p>Let's now take a look at another api. We'll start by looking at just the base URL:  </p>


In [8]:
base_url = 'https://python.zgulde.net'
print(requests.get(base_url).text)

{"api":"/api/v1","help":"/documentation"}



In [9]:
{"api":"/api/v1","help":"/documentation"}


{'api': '/api/v1', 'help': '/documentation'}

<p>This API provides some documentation, so let's make a request so that we can take a look at it.</p>


In [10]:
response = requests.get(base_url + '/documentation')
print(response.json()['payload'])


The API accepts GET requests for all endpoints, where endpoints are prefixed
with

    /api/{version}

Where version is "v1"

Valid endpoints:

- /stores[/{store_id}]
- /items[/{item_id}]
- /sales[/{sale_id}]

All endpoints accept a `page` parameter that can be used to navigate through
the results.



<p>Based on this, let's take a look at the items. We'll make our request, and explore the shape of the response that we get back.</p>


In [11]:
response = requests.get('https://python.zgulde.net/api/v1/items')

data = response.json()
data.keys()

dict_keys(['payload', 'status'])

In [12]:
data['payload'].keys()


dict_keys(['items', 'max_page', 'next_page', 'page', 'previous_page'])

In [13]:
print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

max_page: 3
next_page: /api/v1/items?page=2


<p>Here the response has some built-in properties that tell us how to get to subsequent pages.</p>
<p>Once we've drilled down into the data structure, we'll find that the entire response is a sort of wrapper around the <code>items</code> property:</p>


In [14]:
data['payload']['items'][:2]


[{'item_brand': 'Riceland',
  'item_id': 1,
  'item_name': 'Riceland American Jazmine Rice',
  'item_price': 0.84,
  'item_upc12': '35200264013',
  'item_upc14': '35200264013'},
 {'item_brand': 'Caress',
  'item_id': 2,
  'item_name': 'Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct',
  'item_price': 6.44,
  'item_upc12': '11111065925',
  'item_upc14': '11111065925'}]

<p>We can turn this data into a pandas dataframe:</p>


In [15]:
import pandas as pd

df = pd.DataFrame(data['payload']['items'])
df.head()

,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct,6.44,11111065925,11111065925
2,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixed Berry,2.43,23923330139,23923330139
3,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice Thin Crackers,2.61,759283100036,759283100036


<p>Now that we've gotten the data from the first page, we can extract the data from the next page (as indicated by the API's response), and add it onto our dataframe:</p>


In [16]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()

max_page: 3
next_page: /api/v1/items?page=3


<p>We'll repeat the process one more time:</p>


In [17]:
response = requests.get(base_url + data['payload']['next_page'])
data = response.json()

print('max_page: %s' % data['payload']['max_page'])
print('next_page: %s' % data['payload']['next_page'])

df = pd.concat([df, pd.DataFrame(data['payload']['items'])]).reset_index()

max_page: 3
next_page: None


<p>Now that the API says that the <code>next_page</code> is None, we'll stop making requests, and assume that we have all of the <code>items</code> data.</p>


In [18]:
df.shape

(50, 8)

In [19]:
df

,level_0,index,item_brand,item_id,item_name,item_price,item_upc12,item_upc14
0,0,0.0,Riceland,1,Riceland American Jazmine Rice,0.84,35200264013,35200264013
1,1,1.0,Caress,2,Caress Velvet Bliss Ultra Silkening Beauty Bar - 6 Ct,6.44,11111065925,11111065925
2,2,2.0,Earths Best,3,Earths Best Organic Fruit Yogurt Smoothie Mixed Berry,2.43,23923330139,23923330139
3,3,3.0,Boars Head,4,Boars Head Sliced White American Cheese - 120 Ct,3.14,208528800007,208528800007
4,4,4.0,Back To Nature,5,Back To Nature Gluten Free White Cheddar Rice Thin Crackers,2.61,759283100036,759283100036
...,...,...,...,...,...,...,...,...
45,5,NaN,Mama Marys,46,Pizza Sauce,4.65,35457770664,35457770664
46,6,NaN,Bear Naked,47,Bear Naked Fit Almond Crisp 100 Percent Natural Energy Cereal,7.38,884623708976,884623708976
47,7,NaN,Dove,48,Dove Men + Care Antiperspirant Deodorant Cool Silver,3.72,79400271631,79400271631
48,8,NaN,Easy-off,49,Easy-off Oven Cleaner Lemon Scent,9.54,62338879772,62338879772


<h2 id="further-reading">Further Reading</h2>
<ul>
<li><a href="https://www.dataquest.io/blog/python-api-tutorial/">Using APIs in Python</a></li>
<li><a href="https://www.smashingmagazine.com/2018/01/understanding-using-rest-api/">Understand and using REST APIs</a></li>
</ul>